**DISCLAIMER:** This file was authored in an IDE with Github Copilot enabled.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification')
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification/src')

In [3]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
model = AutoModelForMaskedLM.from_pretrained("dbmdz/bert-base-german-cased")

# Specify the patter to use for this lexical simplifier
bert_pattern = '{original_sentence} Die einfachere Version der vorigen Satzes ist: {sentence_with_complex_word_masked}'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/456 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-german-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from german_bert_lexical_simplifier import GermanBertLexicalSimplifier
german_bert_ls = GermanBertLexicalSimplifier(model, tokenizer, bert_pattern, None)

No exemplars provided, using zero-shot mode.


In [8]:
text = 'Die Linguistik analysiert die Struktur und Entwicklung von Sprachen.'

predicted_tokens = german_bert_ls.generate_substitutions_for('analysiert', text)

print("Predicted tokens:", predicted_tokens)

Predicted tokens: ['analysiert', 'untersucht', 'betrachtet', 'behandelt', 'beschreibt', 'erforscht', 'studiert', 'prüft', 'beobachtet', 'berücksichtigt']
